In [ ]:
import pandas as pd
import nltk.data
import itertools
import random
import numpy as np
import os

In [ ]:
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
SHUFFLE_COUNT = 5
max_seq_len = 10

In [ ]:
data = pd.read_csv("./nips_data/nips_papers.csv")

In [ ]:
abstracts = data['abstract'][data['abstract'] != 'Abstract Missing']

In [ ]:
os.makedirs("./nips_data/original/", exist_ok=True)
os.makedirs("./nips_data/permutations/", exist_ok=True)
os.makedirs("./nips_data/processed/", exist_ok=True)

In [ ]:
for index,abstract in enumerate(abstracts):
    file_name = "./nips_data/original/original_"+str(index)+".csv"
    f = open('%s' % file_name, 'w+')
    abstract = " ".join(abstract.split())
    for line_number,line in enumerate(tokenizer.tokenize(abstract)):
        f.write(str(line_number) + "\t" + line + "\n")
            
    f.close()

In [ ]:
for index in range(len(abstracts)):
    file_name = "./nips_data/original/original_"+str(index)+".csv"
    f = open('%s' % file_name, 'r')
    
    content = f.readlines()
    for i in range(SHUFFLE_COUNT):
        shuffled_file_name = "./nips_data/permutations/original_"+ str(index) +"_permutation_"+str(i)+".csv"
        shuffled_f = open('%s' % shuffled_file_name, 'w+')
        if(i == 0):
            shuffled_content =''.join(content)
        else:
            random.shuffle(content)
            shuffled_content = ''.join(content)
            
        shuffled_f.write(shuffled_content)
        shuffled_f.close()
    
    f.close()

In [ ]:
permutationFiles =  os.listdir("./nips_data/permutations/")
len(permutationFiles)

In [ ]:
permutationData = []
for file in permutationFiles:
    with open("./nips_data/permutations/"+file) as f:
        permutationData.append(f.readlines())

In [ ]:
num_files = len(permutationData)

processedData = np.array([[" " for i in range(max_seq_len)] for i in range(num_files)], dtype = object)
processedDataPerm = np.array([[-1 for i in range(max_seq_len)] for i in range(num_files)])
for i in range(num_files):
    fileData = permutationData[i]
    j = 0
    for line in fileData:
        processedData[i][j] = ''.join(line.split('\t')[1:]).strip()
        
        if(int(line.split('\t')[0]) <= max_seq_len):
            processedDataPerm[i][j] = int(line.split('\t')[0])
            j += 1
        if( j >= max_seq_len ):
            break

In [ ]:
with open('./nips_data/processed/sentences.txt','w+') as f:
    for line in processedData:
        f.write('\t'.join(line).strip() + '\n')

In [ ]:
with open("./nips_data/processed/permutations.txt",'w+') as f:
    for line in processedDataPerm:
        f.write('\t'.join(line.astype(str)).strip() + '\n')